# Gestion des biblothèques

In [ ]:
import pandas as pd
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Pré traitement des données

#### Importation du dataset

In [ ]:
recipes_df = pd.read_csv('../../data/raw/RAW_recipes.csv', on_bad_lines='skip', engine="python")

### Suppression des colonnes inutiles

In [ ]:
recipes_df = recipes_df.drop(['submitted', 'contributor_id', 'tags', 'minutes', 'description', 'nutrition', 'n_steps', 'n_ingredients', 'steps'], axis=1)
recipes_df.info()

#### Suppression des lignes incorrectes

In [ ]:
recipes_df.drop_duplicates()
recipes_df.dropna(inplace=True)

#### Sélection des 20000 premières recettes

In [ ]:
recipes_df = recipes_df.head(20000)

#### Formattage du titre

Les titres sont rentrés manuellement par les utilisateurs. Ils contiennent parfois des espaces en trop, que nous allons chercher à supprimer

In [ ]:
# Retourner le titre avec un seul espace entre les mots.
def clean_recipe_titles(title):
  words = title.split()
  cleaned_title = ' '.join(words)
  return cleaned_title

In [ ]:
recipes_df.name = recipes_df.name.apply(clean_recipe_titles)

#### Nettoyage des ingrédients

In [ ]:
def text_cleaning(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    return text

recipes_df['ingredients'] = recipes_df['ingredients'].apply(lambda x: text_cleaning(x))

#### Affichage des la dimension du dataframe

In [ ]:
recipes_df.shape

#### Vérification du tri

In [ ]:
recipes_df.duplicated().sum()
recipes_df.isnull().sum()

# Filtrage par contenu

### Utilisation de TF-IDF

On accepte 20000 mots de vocabulaire au maximum et on supprime les particules anglaises

In [ ]:
tfidf = TfidfVectorizer(stop_words='english', max_features=20000)

On applique le procession à notre colonne des ingrédients

In [ ]:
tfidf_matrix = tfidf.fit_transform(recipes_df['ingredients'])
tfidf_matrix.shape

### Création du linear kernel

On calcule les angles entre la matrice TF-IDF en ligne et en colonne

In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

### Création des indices

Les indices permettes de rechercher dans le dataframe des recettes. Les id ne correspondant plus à l'index du tableau

In [ ]:
indices = pd.Series(recipes_df.index, index=recipes_df['name']).drop_duplicates()
indices

# Le programme principale

#### Récupération des données

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    try:
      # On récupère la ligne des scores correspondant à la recette
      sim_scores = list(enumerate(cosine_sim[indices[title]]))

      # On trie cette liste du plus grand score au plus faible
      sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

      # On retient que les 5 premiers scores (on a gardé la recette originelle pour comparer)
      sim_scores = sim_scores[0:6]

      # Transformation de la liste en dataframe
      food_indices = [i[0] for i in sim_scores]

      df_reco = pd.DataFrame({'index': food_indices, 'similarity': [i[1] for i in sim_scores]})
      df_reco = pd.merge(df_reco, recipes_df[['name', 'ingredients']], left_on='index', right_index=True)
      df_reco = df_reco[['name', 'ingredients', 'similarity']]

      return df_reco

    except:
      print("La recette n'existe pas dans la base de données")

### Lancement du programme

In [ ]:
recette = input("Entrez le nom d'une recette : (ex. fool the meat eaters chili)")
print_df = get_recommendations(recette)

if print_df is not None:
   display(print_df.head())